Pymaceuticals Inc.


Analysis

We obtain two data set:
First data set include Mouse_Metada Data consist of ID,Se, Age,Weight, and Drug information.Second data set consists of treatment information for the each mice with size of tumor, medication, timepoints, and progress(regress) of tumor.
We started our analysis combining two data sets together to determine any relationship between variables in the data.
-The data has no biases in the sex of the mice. Female and male mice population is close to each other with male population 1.92% higher. The variance in insignificant.
-The sttistical analysis of each drug regimen demonstrated similar distrubition. based standard deviation we can conclude that the data clustered around mean and median. Other than Infubinol drug regimen tehre is no outliers in the data.
-Total number of timepoints for all mice tested for each drug regimen show also similar pattern.
-when the relationship between # of times mice tested and Tumor volume tested, it conluded that the relationship is linear and negatively correlated. it indicates that at the first test timepoint the tumor volume/mass is high, but tumor reduces volume as medication is applied and the tests are repeated. This concludes that all the other variables being constant, for Drug Regimen Capomulin Treatment is showing progress for mouse "s185".
-The relationship between mice weight and tumor volume variables also shows linear relationship. Contrary to timepoint the relationship is positive. When tumor volume increases the mouse weight  also increases. 

 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import sklearn.datasets as dta
from scipy.stats import linregress
# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_df=pd.merge(mouse_metadata, study_results, on=["Mouse ID"])

# Display the data table for preview
mouse_df.head()

In [ ]:
# Checking the number of mice.
mouse_df_uniq=mouse_df["Mouse ID"].nunique()
mouse_df_uniq

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouse_df_timepoint=mouse_df.loc[mouse_df.duplicated(subset=["Mouse ID",'Timepoint']),"Mouse ID"].unique()

duplicated_list=mouse_df_timepoint[0]

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicated_data=mouse_df.loc[mouse_df["Mouse ID"]== duplicated_list]
duplicated_data

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cleaned_data=mouse_df.loc[mouse_df["Mouse ID"]!= duplicated_list]
cleaned_data

In [ ]:
# Checking the number of mice in the clean DataFrame.
cleaned_data_count=cleaned_data["Mouse ID"].nunique()
cleaned_data_count

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
summary_stat_mean =cleaned_data.groupby(["Drug Regimen"]).mean()["Tumor Volume (mm3)"]
summary_stat_median =cleaned_data.groupby(["Drug Regimen"]).median()["Tumor Volume (mm3)"]
summary_stat_std =cleaned_data.groupby(["Drug Regimen"]).std()["Tumor Volume (mm3)"]
summary_stat_var =cleaned_data.groupby(["Drug Regimen"]).var()["Tumor Volume (mm3)"]
summary_stat_sem =cleaned_data.groupby(["Drug Regimen"]).sem()["Tumor Volume (mm3)"]
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
summary_stat_df=pd.DataFrame({
    "mean":summary_stat_mean,
    "median":summary_stat_median,
    "standard_dev":summary_stat_std,
    "variance":summary_stat_var,
    "sem":summary_stat_sem
})
summary_stat_df

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen
summary_stat_mean =cleaned_data.groupby(["Drug Regimen"]).mean()["Tumor Volume (mm3)"]
summary_stat_median =cleaned_data.groupby(["Drug Regimen"]).median()["Tumor Volume (mm3)"]
summary_stat_std =cleaned_data.groupby(["Drug Regimen"]).std()["Tumor Volume (mm3)"]
summary_stat_var =cleaned_data.groupby(["Drug Regimen"]).var()["Tumor Volume (mm3)"]
summary_stat_sem =cleaned_data.groupby(["Drug Regimen"]).sem()["Tumor Volume (mm3)"]
# Using the aggregation method, produce the same summary statistics in a single line.
summary_stat_agg =cleaned_data.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].agg(['mean', 'median', 'var', 'std', 'sem'])
summary_stat_agg

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.
timepoint_df=cleaned_data['Drug Regimen'].value_counts()
timepoint_df
cleaned_data.groupby('Drug Regimen').count().plot(kind='bar', legend=False, color='g',alpha=0.7,figsize=(7,4))
plt.title('# of Timepoint by Drug Regimen')
plt.xlabel('Drug Regimen')
plt.ylabel('Timepoint')
plt.ylim(0,235)
plt.show()

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.
x_axis = timepoint_df.index.values
y_axis = timepoint_df.values
plt.bar(x_axis, y_axis, color='g', alpha=0.6, align="edge",)
plt.xticks(rotation=45, rotation_mode="anchor", ha="right", wrap=True)
plt.ylim(0,235)
plt.show()


In [ ]:
gender_df = cleaned_data["Sex"].value_counts()
gender_df

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
gender_df.plot(kind= "pie",autopct= "%1.2f%%")
plt.title("Female versus Male Mice")
plt.show()
plt.tight_layout()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
y=gender_df

mylabels =gender_df.index
mysizes = gender_df
myexplode=[0.05, 0]

plt.pie(y,labels = mylabels,  explode = myexplode, shadow = True, autopct="%1.2f%%")
plt.title('Female versus Male Mice')
plt.ylabel('Sex')
plt.show()
plt.tight_layout()


## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin
list=["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]
new_data = cleaned_data[cleaned_data["Drug Regimen"].isin(list)]
# Start by getting the last (greatest) timepoint for each mouse
new_data_df=new_data.groupby('Mouse ID').max()['Timepoint']

new_data_df_last = pd.DataFrame(new_data_df)
# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
new_data_mg=pd.merge(new_data_df_last, cleaned_data, on=['Mouse ID',"Timepoint"])
new_data_mg

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatment_list=["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]
 
# Create empty list to fill with tumor vol data (for plotting)
tumor_vol_list=[]

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatment_list:   
    # Locate the rows which contain mice on each drug and get the tumor volumes
    final_tumor_vol=new_data_mg.loc[new_data_mg["Drug Regimen"] == drug, 'Tumor Volume (mm3)']
    
    # add subset 
    tumor_vol_list.append(final_tumor_vol)
    
    # Determine outliers using upper and lower bounds
    quartiles = final_tumor_vol.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    outliers=final_tumor_vol.loc[(final_tumor_vol< lower_bound)|(final_tumor_vol>upper_bound)]
    print(f"{drug}'s potential outliers: {outliers}")
    drug1="Capomulin"
    drug2="Ramicane"
    drug3= "Infubinol"
    drug4="Ceftamin"
    cap_vol=new_data_mg.loc[new_data_mg["Drug Regimen"] == drug1, 'Tumor Volume (mm3)']
    ram_vol=new_data_mg.loc[new_data_mg["Drug Regimen"] == drug2, 'Tumor Volume (mm3)']
    infu_vol=new_data_mg.loc[new_data_mg["Drug Regimen"] == drug3, 'Tumor Volume (mm3)']
    cef_vol=new_data_mg.loc[new_data_mg["Drug Regimen"] == drug4, 'Tumor Volume (mm3)']

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group
myflierprops = dict(marker='o', markerfacecolor='green', markersize=12,linestyle='none')
plt.boxplot([cap_vol, ram_vol, infu_vol, cef_vol],labels=['Capomulin','Ramicane','Infubinol','Ceftamin'],flierprops=myflierprops,showmeans=True,meanline=True)
plt.ylabel('Final Tumor Vol (mm3)')
plt.title('The Distrubution for each Treatment Group')
plt.ylim(10,75)

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin
new_cap_data = cleaned_data[cleaned_data["Mouse ID"] =='s185']
new_cap_data



In [ ]:
y=new_cap_data['Tumor Volume (mm3)']
x=new_cap_data['Timepoint']
Mouse_ID="s185"
plt.plot(x, y, 'go--', linewidth=2, markersize=12)
plt.ylabel('Tumor Vol (mm3)')
plt.xlabel('Timepoint')
plt.title('Capomulin Treatment Progress for s185')
plt.grid(visible=None, which='major', axis='x')
plt.xlim(-0.5,46)
plt.ylim(20,46)
plt.show()

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
new_cap_df = cleaned_data[cleaned_data["Drug Regimen"] =='Capomulin']
new_cap_df
average_df=new_cap_df.groupby(['Mouse ID']).mean()
average_df
x_values=average_df['Weight (g)']
y_values=average_df['Tumor Volume (mm3)']
plt.scatter(x_values,y_values, color="green")
plt.xlabel('Weight (g)',fontsize =14)
plt.ylabel('Average Tumor Volume (mm3)')
plt.title('Average Tumor Volume vs. Mouse Weight')
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
corr_df = st.pearsonr(x_values,y_values)
corr_df
print(f"The correlation coefficient between mouse weight and the average tumor volume is {round(corr_df[0],2)}")



In [ ]:
plt.scatter(x_values,y_values, color="green")
x_values=average_df['Weight (g)']
y_values=average_df['Tumor Volume (mm3)']
(slope, intercept, rvalue,pvalue,std_err) = linregress(x_values,y_values)
slope=round(slope,2)
intercept=round(intercept,2)
line_str=f"y={slope}x+{intercept}"
plt.annotate(line_str,(60,36), fontsize=14)
predicted_value=x_values*slope+intercept
plt.plot(x_values,predicted_value,"r-")
print(rvalue**2)
plt.xlabel("Weight (g)")
plt.ylabel("Tumor Volume (mm3)")
plt.title("Mouse Weight and Average Tumor Volume for the Capomulin")
plt.show()